In [2]:
import os
print("當前工作目錄:", os.getcwd())
print("這裡有哪些檔案:", os.listdir())
#os.chdir('/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_CNN')
print("當前工作目錄:", os.getcwd())

當前工作目錄: /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/compare_model
這裡有哪些檔案: ['best_model_tmp.pth', '__init__.py', 'csidataset.py', 'data_loader.py', 'best_model_tmp_24.pth', 'repeat', 'FIFS.ipynb', 'DeepFi.ipynb', 'models_save', '2.4Grssi_csi.ipynb', '2.4GVS5G.ipynb', '__pycache__', 'repeat_2_4', 'reapeat_2_4', '.git']
當前工作目錄: /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/compare_model


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import sys, os
import pandas as pd
import matplotlib.pyplot as plt

from csidataset import *
import data_loader
from data_loader import *
sys.path.append("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool")
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#CSI

In [ ]:
base_path2 = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/localization_phone/1207_phone/20MHz/csv/"


In [5]:
reference_points2 = {}

spacing = 0.6  # 每隔 0.6m

for ref_id, coord in data_loader.COORDINATES.items():
    folder_path2 = os.path.join(base_path2, f"reference_point_{ref_id}.xlsx")
    reference_points2[folder_path2] = (ref_id, coord)

data, rp_labels, coord_labels = load_data(reference_points2)
amp_data = np.array(data.iloc[:, :48])  
phase_data = np.array(data.iloc[:, 48:-2]) 

encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = encoder.fit_transform(np.array(rp_labels).reshape(-1, 1))

In [6]:
csi = data
csi["Label"] = rp_labels

In [7]:
csi

,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,Label
0,511.173160,510.098030,508.550882,501.722035,501.897400,510.107832,524.969523,523.519818,535.526843,543.033148,...,-0.407695,-0.545071,-0.689205,-0.863163,-1.231418,-1.472350,-1.706611,-1.706611,-2.331028,1
1,461.009761,463.605436,450.249931,456.326637,442.376536,462.497568,486.054524,571.521653,592.497257,639.728849,...,-12.169046,-12.616624,-13.024942,-13.451668,-14.402662,-14.932274,-15.487349,-16.046496,-16.638071,1
2,503.062620,512.621693,492.853934,522.770504,488.331854,509.062864,512.665583,511.100773,514.117691,537.461627,...,-9.871651,-10.169290,-10.477410,-10.810423,-11.528382,-11.913213,-12.331859,-12.761813,-13.264725,1
3,476.320270,476.589971,461.971861,452.491989,441.021541,470.340302,500.372861,580.135329,589.433626,641.935355,...,-10.537019,-10.741736,-10.944798,-11.154691,-11.657091,-11.915701,-12.219685,-12.538455,-12.911452,1
4,712.789590,714.217054,692.699791,705.226914,692.223230,711.961375,717.446862,717.948466,721.388938,742.038409,...,-5.962972,-6.141288,-6.310889,-6.508097,-6.938208,-7.193479,-7.477194,-7.801464,-8.180524,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,825.496820,798.105883,779.602463,763.300727,708.204067,655.640908,643.680822,605.607133,588.126687,566.083033,...,-0.371129,-0.539062,-0.674741,-0.855087,-1.219859,-1.456186,-1.717840,-2.003850,-2.354260,49
24420,690.976121,676.932050,625.166378,601.917768,562.099635,549.105636,561.968860,595.372992,590.762220,615.709347,...,-1.728284,-1.882076,-2.023848,-2.153251,-2.511025,-2.743070,-2.999181,-3.269016,-3.575449,49
24421,840.615251,815.965686,766.724201,753.095611,697.585120,674.439026,665.009022,617.778277,579.118295,563.181143,...,-5.944020,-6.160811,-6.357044,-6.606240,-7.108562,-7.387272,-7.698331,-8.064308,-8.479586,49
24422,689.890571,674.146127,639.413794,609.915568,568.810162,546.962522,558.162163,598.030100,598.305942,608.247483,...,-10.075983,-10.406599,-10.684149,-10.983091,-11.665294,-12.061235,-12.490864,-12.929455,-13.389846,49


In [8]:
# Load data RSSI

label_to_coordinates_swapped = {
    "1-1":  (0, 0),   "1-2":  (0, 0.6),   "1-3":  (0, 1.2),   "1-4":  (0, 1.8),
    "1-5":  (0, 2.4), "1-6":  (0, 3.0),   "1-7":  (0, 3.6),   "1-8":  (0, 4.2),
    "1-9":  (0, 4.8), "1-10": (0, 5.4),   "1-11": (0, 6.0),

    "2-1":  (0.6, 0),  "2-11": (0.6, 6.0),

    "3-1":  (1.2, 0),  "3-11": (1.2, 6.0),

    "4-1":  (1.8, 0),  "4-11": (1.8, 6.0),

    "5-1":  (2.4, 0),  "5-11": (2.4, 6.0),

    "6-1":  (3.0, 0),  "6-2":  (3.0, 0.6),  "6-3":  (3.0, 1.2),  "6-4":  (3.0, 1.8),
    "6-5":  (3.0, 2.4), "6-6":  (3.0, 3.0),  "6-7":  (3.0, 3.6),  "6-8":  (3.0, 4.2),
    "6-9":  (3.0, 4.8), "6-10": (3.0, 5.4),  "6-11": (3.0, 6.0),

    "7-1":  (3.6, 0),  "7-11": (3.6, 6.0),

    "8-1":  (4.2, 0),  "8-11": (4.2, 6.0),

    "9-1":  (4.8, 0),  "9-11": (4.8, 6.0),

    "10-1": (5.4, 0),  "10-11": (5.4, 6.0),

    "11-1": (6.0, 0),  "11-2": (6.0, 0.6), "11-3": (6.0, 1.2), "11-4": (6.0, 1.8),
    "11-5": (6.0, 2.4),"11-6": (6.0, 3.0), "11-7": (6.0, 3.6), "11-8": (6.0, 4.2),
    "11-9": (6.0, 4.8),"11-10":(6.0, 5.4), "11-11":(6.0, 6.0)
}


coordinates = {
        1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
        36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
        2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
        6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
        12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
        16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
        20: (5.4, 6.0), 21: (6.0, 6.0),
        22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
        26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
        41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
        44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
        47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
    }
coordinate_to_label2 = {value: key for key, value in coordinates.items()}

# label_to_coordinates:  "1-1" -> (0, 0)
# coordinates:          1      -> (0, 0)

# 1) 先建立一個「座標 -> 數字」的反轉字典
coord_to_num = {value: key for key, value in coordinates.items()}
# coord_to_num[(0,0)] = 1,  coord_to_num[(0.6,0)] = 40, ...

# 2) 建立 "1-1" -> 整數標籤 的 map
label_str_to_num = {}
for str_label, coord in label_to_coordinates_swapped.items():
    if coord in coord_to_num:  # 確保在座標字典中找得到
        label_str_to_num[str_label] = coord_to_num[coord]

In [8]:
# RSSI

In [10]:
rssi_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/rssi/rssi_for_4AP.csv"
df_final = pd.read_csv(rssi_path)

In [11]:
import pandas as pd

# === 假設以下兩個 DataFrame 已經就緒 ===
# df_rssi: 包含 ["Label", ...] + RSSI 欄位
# df_csi:  包含 ["Label", ...] + CSI 欄位

# 取得兩邊都出現的所有 Label
common_labels = sorted(set(df_final["Label"].unique()) & set(csi["Label"].unique()))

merged_list = []

for lb in common_labels:
    # 1. 取出該 Label 的子集
    sub_rssi = df_final[df_final["Label"] == lb].reset_index(drop=True)
    sub_csi  = csi[csi["Label"] == lb].reset_index(drop=True)
    
    # 2. 檢查兩者長度是否一致
    len_rssi = len(sub_rssi)
    len_csi  = len(sub_csi)
    
    if len_rssi != len_csi:
        print(f"警告：Label={lb} 在 df_rssi 有 {len_rssi} 筆, df_csi 有 {len_csi} 筆，長度不同。")
        # 你可以選擇：略過、或僅取最短長度、或補齊…
        # 這邊示範「取最小長度」來對齊
        min_len = min(len_rssi, len_csi)
        sub_rssi = sub_rssi.iloc[:min_len].reset_index(drop=True)
        sub_csi  = sub_csi.iloc[:min_len].reset_index(drop=True)
    
    # 3. 逐列「貼合」兩個子 DataFrame (axis=1 即左右合併)
    #    如果不想留重複的 Label 欄位，可以先丟掉 sub_csi 的 Label 欄位
    sub_csi_dropped = sub_csi.drop(columns=["Label"])
    
    # 合併
    sub_merged = pd.concat([sub_rssi, sub_csi_dropped], axis=1)
    
    # 4. 把合併後的子 DataFrame 放進清單
    merged_list.append(sub_merged)

# 5. 最後把所有 Label 的結果上下堆起來
df_merged_all = pd.concat(merged_list, ignore_index=True)

# 檢查
print(df_merged_all.shape)
print(df_merged_all.head())




 

警告：Label=1 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=2 在 df_rssi 有 500 筆, df_csi 有 497 筆，長度不同。
警告：Label=3 在 df_rssi 有 500 筆, df_csi 有 497 筆，長度不同。
警告：Label=5 在 df_rssi 有 500 筆, df_csi 有 498 筆，長度不同。
警告：Label=6 在 df_rssi 有 500 筆, df_csi 有 497 筆，長度不同。
警告：Label=7 在 df_rssi 有 500 筆, df_csi 有 497 筆，長度不同。
警告：Label=8 在 df_rssi 有 500 筆, df_csi 有 498 筆，長度不同。
警告：Label=9 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=11 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=12 在 df_rssi 有 500 筆, df_csi 有 492 筆，長度不同。
警告：Label=13 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=14 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=15 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=19 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=22 在 df_rssi 有 500 筆, df_csi 有 498 筆，長度不同。
警告：Label=25 在 df_rssi 有 500 筆, df_csi 有 498 筆，長度不同。
警告：Label=29 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=33 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=35 在 df_rssi 有 500 筆, df_csi 有 499 筆，長度不同。
警告：Label=37 在 df_rss

In [12]:
df_merged_all.shape

(24424, 101)

In [13]:
df_merged_all

,Label,AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,0,1,2,3,4,...,86,87,88,89,90,91,92,93,94,95
0,1,-62.0,-72.0,-65.0,-54.0,511.173160,510.098030,508.550882,501.722035,501.897400,...,-0.280435,-0.407695,-0.545071,-0.689205,-0.863163,-1.231418,-1.472350,-1.706611,-1.706611,-2.331028
1,1,-61.0,-73.0,-66.0,-56.0,461.009761,463.605436,450.249931,456.326637,442.376536,...,-11.725221,-12.169046,-12.616624,-13.024942,-13.451668,-14.402662,-14.932274,-15.487349,-16.046496,-16.638071
2,1,-62.0,-72.0,-66.0,-55.0,503.062620,512.621693,492.853934,522.770504,488.331854,...,-9.569820,-9.871651,-10.169290,-10.477410,-10.810423,-11.528382,-11.913213,-12.331859,-12.761813,-13.264725
3,1,-61.0,-73.0,-66.0,-55.0,476.320270,476.589971,461.971861,452.491989,441.021541,...,-10.340610,-10.537019,-10.741736,-10.944798,-11.154691,-11.657091,-11.915701,-12.219685,-12.538455,-12.911452
4,1,-54.0,-72.0,-67.0,-55.0,712.789590,714.217054,692.699791,705.226914,692.223230,...,-5.812728,-5.962972,-6.141288,-6.310889,-6.508097,-6.938208,-7.193479,-7.477194,-7.801464,-8.180524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,49,-54.0,-64.0,-56.0,-58.0,825.496820,798.105883,779.602463,763.300727,708.204067,...,-0.250627,-0.371129,-0.539062,-0.674741,-0.855087,-1.219859,-1.456186,-1.717840,-2.003850,-2.354260
24420,49,-55.0,-65.0,-56.0,-57.0,690.976121,676.932050,625.166378,601.917768,562.099635,...,-1.597164,-1.728284,-1.882076,-2.023848,-2.153251,-2.511025,-2.743070,-2.999181,-3.269016,-3.575449
24421,49,-53.0,-63.0,-53.0,-56.0,840.615251,815.965686,766.724201,753.095611,697.585120,...,-5.729949,-5.944020,-6.160811,-6.357044,-6.606240,-7.108562,-7.387272,-7.698331,-8.064308,-8.479586
24422,49,-53.0,-64.0,-55.0,-59.0,689.890571,674.146127,639.413794,609.915568,568.810162,...,-9.775735,-10.075983,-10.406599,-10.684149,-10.983091,-11.665294,-12.061235,-12.490864,-12.929455,-13.389846


In [14]:
rssi_columns = ['AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi']
df_merged_all[rssi_columns] = df_merged_all[rssi_columns].fillna(0)

In [15]:
rp_labels = df_merged_all['Label']
csi = df_merged_all.drop(columns=['Label', 'AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi'])
amp = csi.iloc[:,:48]
phase = csi.iloc[:,48:-2]
rssi = df_merged_all[['AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi']]

In [16]:
print("csi shape:", rssi.shape)
print("amp shape:", amp.shape)

csi shape: (24424, 4)
amp shape: (24424, 48)


In [17]:
rssi

,AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi
0,-62.0,-72.0,-65.0,-54.0
1,-61.0,-73.0,-66.0,-56.0
2,-62.0,-72.0,-66.0,-55.0
3,-61.0,-73.0,-66.0,-55.0
4,-54.0,-72.0,-67.0,-55.0
...,...,...,...,...
24419,-54.0,-64.0,-56.0,-58.0
24420,-55.0,-65.0,-56.0,-57.0
24421,-53.0,-63.0,-53.0,-56.0
24422,-53.0,-64.0,-55.0,-59.0


In [18]:
amp

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,511.173160,510.098030,508.550882,501.722035,501.897400,510.107832,524.969523,523.519818,535.526843,543.033148,...,643.123627,655.746902,626.834109,647.876531,655.347236,645.837441,651.152824,627.781013,627.781013,618.207085
1,461.009761,463.605436,450.249931,456.326637,442.376536,462.497568,486.054524,571.521653,592.497257,639.728849,...,774.070410,728.771569,676.854490,616.715494,609.737649,579.296988,582.683448,557.512332,584.154945,568.802250
2,503.062620,512.621693,492.853934,522.770504,488.331854,509.062864,512.665583,511.100773,514.117691,537.461627,...,629.610991,613.216112,622.774437,631.957277,624.676716,608.323105,600.628005,602.491494,602.469916,601.905308
3,476.320270,476.589971,461.971861,452.491989,441.021541,470.340302,500.372861,580.135329,589.433626,641.935355,...,750.249292,702.581668,657.054792,610.787197,599.574015,555.090083,558.014337,532.693157,573.223342,555.763439
4,712.789590,714.217054,692.699791,705.226914,692.223230,711.961375,717.446862,717.948466,721.388938,742.038409,...,853.750549,822.825619,813.172798,830.318614,834.005396,815.854154,811.717315,799.052564,800.103118,791.843419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24419,825.496820,798.105883,779.602463,763.300727,708.204067,655.640908,643.680822,605.607133,588.126687,566.083033,...,858.832347,871.320836,884.419018,941.259263,964.713947,986.101922,1005.597335,1044.269122,1081.868754,1096.724669
24420,690.976121,676.932050,625.166378,601.917768,562.099635,549.105636,561.968860,595.372992,590.762220,615.709347,...,910.316429,879.889198,845.639403,801.898996,799.889992,788.665962,824.621125,824.345195,889.209199,915.852608
24421,840.615251,815.965686,766.724201,753.095611,697.585120,674.439026,665.009022,617.778277,579.118295,563.181143,...,869.740766,871.655895,876.555189,921.512344,970.187611,1008.432943,1022.335072,1038.555246,1092.065932,1128.780758
24422,689.890571,674.146127,639.413794,609.915568,568.810162,546.962522,558.162163,598.030100,598.305942,608.247483,...,898.786404,882.626195,855.070757,796.303962,801.062420,808.677933,841.038049,848.417350,895.373107,891.600808


In [19]:
from sklearn.model_selection import train_test_split

# 取得原始的 index
indices = amp.index

# 先切分 train 與 temp (train: 70%, temp: 30%)
train_idx, temp_idx = train_test_split(indices, test_size=0.3, random_state=42)

# 接著切分 temp 為 validation 與 test (各 10% 與 20%，因為 1/3 的 30% 為 10%)
val_idx, test_idx = train_test_split(temp_idx, test_size=1/3, random_state=42)

# 使用 .loc 根據切分的 index 取得各部分資料
amp_train = amp.loc[train_idx]
amp_val   = amp.loc[val_idx]
amp_test  = amp.loc[test_idx]

rssi_train = rssi.loc[train_idx]
rssi_val   = rssi.loc[val_idx]
rssi_test  = rssi.loc[test_idx]

# 假設 one_hot_labels 也是與 amp 同 index 的 Series 或 DataFrame
y_train = rp_labels.loc[train_idx]
y_val   = rp_labels.loc[val_idx]
y_test  = rp_labels.loc[test_idx]


In [20]:
encoder = OneHotEncoder(sparse_output=False)
y_train_oh = encoder.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_oh = encoder.transform(np.array(y_val).reshape(-1, 1))
y_test_oh = encoder.transform(np.array(y_test).reshape(-1, 1))

In [21]:
# 創建 Dataset 和 DataLoader

batch_size = 32

train_dataset = CSIRSSIDataset(np.array(amp_train), np.array(rssi_train), y_train_oh, augment=True, csi_noise_std=0.00, rssi_mask_prob=0.05)
val_dataset = CSIRSSIDataset(np.array(amp_val),np.array(rssi_val), y_val_oh)
test_dataset = CSIRSSIDataset(np.array(amp_test), np.array(rssi_test), y_test_oh)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [22]:
for amp_inputs, rssi_inputs, labels in train_loader:
        print(labels)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ...

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import os

# 假設模型已經定義好
class CSIRSSI_Classifier_BN(nn.Module):
    def __init__(self, num_classes=49, rssi_dim=4):
        super(CSIRSSI_Classifier_BN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.flatten_dim = 128 * 12
        self.fc1   = nn.Linear(self.flatten_dim, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2   = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.3)

        self.fc_rssi1 = nn.Linear(rssi_dim, 128)
        self.bn_rssi1 = nn.BatchNorm1d(128)
        self.dropout_rssi1 = nn.Dropout(0.5)
        self.fc_rssi2 = nn.Linear(128, 32)
        self.bn_rssi2 = nn.BatchNorm1d(32)
        self.dropout_rssi2 = nn.Dropout(0.5)

        self.fc_fusion = nn.Linear(64 + 32, 64)
        self.dropout_fusion = nn.Dropout(0.3)
        self.fc_class = nn.Linear(64, num_classes)

    def forward(self, csi_input, rssi_input):
        if csi_input.dim() == 2:
            csi_input = csi_input.unsqueeze(1)
        x = F.relu(self.bn1(self.conv1(csi_input)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        csi_feat = F.relu(self.fc2(x))
        csi_feat = self.dropout2(csi_feat)

        rssi_feat = F.relu(self.bn_rssi1(self.fc_rssi1(rssi_input)))
        rssi_feat = self.dropout_rssi1(rssi_feat)
        rssi_feat = F.relu(self.bn_rssi2(self.fc_rssi2(rssi_feat)))
        rssi_feat = self.dropout_rssi2(rssi_feat)

        fusion = torch.cat([csi_feat, rssi_feat], dim=1)
        fusion = F.relu(self.fc_fusion(fusion))
        fusion = self.dropout_fusion(fusion)

        class_out = self.fc_class(fusion)
        return class_out

    
    def extract_embedding(self, csi_input, rssi_input):
        if csi_input.dim() == 2:
            csi_input = csi_input.unsqueeze(1)
        x = F.relu(self.bn1(self.conv1(csi_input)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        csi_feat = F.relu(self.fc2(x))
        rssi_feat = F.relu(self.bn_rssi1(self.fc_rssi1(rssi_input)))
        rssi_feat = F.relu(self.bn_rssi2(self.fc_rssi2(rssi_feat)))
        fusion = torch.cat([csi_feat, rssi_feat], dim=1)
        fusion = F.relu(self.fc_fusion(fusion))
        return fusion




In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os

# ===== 你的模型 CSIRSSI_Classifier_BN 定義在這裡（略） =====

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def compute_mean_distance_error(y_true, y_pred, coordinates):
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label not in coordinates or pred_label not in coordinates:
            print(f"Label {true_label} or {pred_label} not in coordinates.")
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        error = np.linalg.norm(pred_coord - true_coord)
        errors.append(error)
    return np.mean(errors), errors

COORDINATES = {
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

num_runs = 10
epochs = 200
patience = 20

test_accs = []
test_mdes = []
all_run_errors = []
os.makedirs("repeat/00", exist_ok=True)

for run in range(1, num_runs + 1):
    print(f"\n=== Run {run}/{num_runs} ===")

    model = CSIRSSI_Classifier_BN(num_classes=49).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15)
    criterion_cls = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    counter = 0

    for epoch in range(epochs):
        model.train()
        for csi_inputs, rssi_inputs, labels in train_loader:
            csi_inputs, rssi_inputs, labels = csi_inputs.to(device), rssi_inputs.to(device), labels.to(device)
            target_class = torch.argmax(labels, dim=1)

            optimizer.zero_grad()
            class_out = model(csi_inputs, rssi_inputs)
            loss = criterion_cls(class_out, target_class)
            loss.backward()
            optimizer.step()

        # 驗證
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for csi_inputs, rssi_inputs, labels in val_loader:
                csi_inputs, rssi_inputs, labels = csi_inputs.to(device), rssi_inputs.to(device), labels.to(device)
                target_class = torch.argmax(labels, dim=1)
                class_out = model(csi_inputs, rssi_inputs)
                val_loss += criterion_cls(class_out, target_class).item() * csi_inputs.size(0)
        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), "repeat_2_4/00/best_csirssi_cls_temp.pth")
        else:
            counter += 1
            if counter >= patience:
                break

    # 測試（只載入暫存的最佳 val loss 權重）
    model.load_state_dict(torch.load("repeat_2_4/00/best_csirssi_cls_temp.pth"))
    model.eval()
    all_true, all_pred = [], []
    with torch.no_grad():
        for csi_inputs, rssi_inputs, labels in test_loader:
            csi_inputs, rssi_inputs, labels = csi_inputs.to(device), rssi_inputs.to(device), labels.to(device)
            target_class = torch.argmax(labels, dim=1)
            class_out = model(csi_inputs, rssi_inputs)
            pred = torch.argmax(class_out, dim=1)
            all_pred.extend(pred.cpu().numpy())
            all_true.extend(target_class.cpu().numpy())

    y_true = np.array(all_true) + 1
    y_pred = np.array(all_pred) + 1
    acc = 100 * np.mean(y_true == y_pred)
    mde, errors = compute_mean_distance_error(y_true, y_pred, COORDINATES)

    all_run_errors.append(errors)

    test_accs.append(acc)
    test_mdes.append(mde)
    print(f"✅ Run {run}: Acc = {acc:.2f}%, MDE = {mde:.4f}")

# 儲存 summary 結果
os.makedirs("repeat/00", exist_ok=True)
df = pd.DataFrame({
    "run": list(range(1, num_runs+1)),
    "accuracy": test_accs,
    "mde": test_mdes
})
df.to_csv("repeat_2_4/00/csirssicls_results00_b.csv", index=False)


# 儲存所有 error（長條格式）
error_records = []
for run_idx, errors in enumerate(all_run_errors):
    for sample_idx, e in enumerate(errors):
        error_records.append({
            "run": run_idx + 1,
            "sample_idx": sample_idx + 1,
            "error": e
        })
df_errors = pd.DataFrame(error_records)
df_errors.to_csv("repeat_2_4/00/csirssicls_all_errors00_b.csv", index=False)




=== Run 1/10 ===


KeyboardInterrupt: 

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary  # torchinfo 可用來顯示多輸入模型摘要
import matplotlib.pyplot as plt

# -----------------------
# 模型定義：CSIRSSI_DualHead
# -----------------------
class CSIRSSI_DualHead_BN(nn.Module):
    def __init__(self, num_classes=49, rssi_dim=4):
        super(CSIRSSI_DualHead_BN, self).__init__()
        # ---- CSI 分支 (CNN) ---
        # 假設輸入 CSI shape 為 (batch, 1, 48)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        
        # 輸入長度 48 → 經過兩次 pooling → 48/2=24，再 24/2=12
        self.flatten_dim = 128 * 12
        
        self.fc1   = nn.Linear(self.flatten_dim, 128)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2   = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.3)
        # 此時 CSI 分支輸出 64 維特徵
        
        # ---- RSSI 分支 (MLP) ----
        self.fc_rssi1 = nn.Linear(rssi_dim, 128)
        self.bn_rssi1 = nn.BatchNorm1d(128)
        self.dropout_rssi1 = nn.Dropout(0.5)
        self.fc_rssi2 = nn.Linear(128, 32)
        self.bn_rssi2 = nn.BatchNorm1d(32)
        self.dropout_rssi2 = nn.Dropout(0.5)
        # RSSI 分支輸出 32 維特徵
        
        # ---- 融合層 ----
        # 將 CSI (64-d) 與 RSSI (32-d) 連接 → 96-d
        self.fc_fusion = nn.Linear(64+32, 64)
        self.dropout_fusion = nn.Dropout(0.3)
        
        # ---- 雙輸出頭 ----
        # 分類頭：輸出 num_classes 個類別的 logits
        self.fc_class = nn.Linear(64, num_classes)
        # 回歸頭：輸出 2 個數值 (X, Y)
        self.fc_reg = nn.Linear(64, 2)
    
    def forward(self, csi_input, rssi_input):
        # CSI 分支
        # 如果輸入為 (batch, 48)，則擴展成 (batch, 1, 48)
        if csi_input.dim() == 2:
            csi_input = csi_input.unsqueeze(1)
        x = F.relu(self.bn1(self.conv1(csi_input)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)  # flatten → (batch, flatten_dim)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        csi_feat = F.relu(self.fc2(x))
        csi_feat = self.dropout2(csi_feat)
        
        # RSSI 分支
        rssi_feat = F.relu(self.bn_rssi1(self.fc_rssi1(rssi_input)))
        rssi_feat = self.dropout_rssi1(rssi_feat)
        rssi_feat = F.relu(self.bn_rssi2(self.fc_rssi2(rssi_feat)))
        rssi_feat = self.dropout_rssi2(rssi_feat)
        
        # 融合特徵
        fusion = torch.cat([csi_feat, rssi_feat], dim=1)  # (batch, 96)
        fusion = F.relu(self.fc_fusion(fusion))
        fusion = self.dropout_fusion(fusion)
        
        # 雙輸出頭
        class_out = self.fc_class(fusion)  # 分類輸出 (batch, num_classes)
        reg_out = self.fc_reg(fusion)       # 回歸輸出 (batch, 2)
        
        return class_out, reg_out

    # def extract_embedding(self, csi_input, rssi_input):
    #     if csi_input.dim() == 2:
    #         csi_input = csi_input.unsqueeze(1)
    #     x = F.relu(self.bn1(self.conv1(csi_input)))
    #     x = self.pool1(x)
    #     x = F.relu(self.bn2(self.conv2(x)))
    #     x = self.pool2(x)
    #     x = x.view(x.size(0), -1)
    #     x = F.relu(self.fc1(x))
    #     csi_feat = F.relu(self.fc2(x))
    #     rssi_feat = F.relu(self.bn_rssi1(self.fc_rssi1(rssi_input)))
    #     rssi_feat = F.relu(self.bn_rssi2(self.fc_rssi2(rssi_feat)))
    #     fusion = torch.cat([csi_feat, rssi_feat], dim=1)
    #     fusion = F.relu(self.fc_fusion(fusion))
    #     return fusion

# -----------------------
# 模型初始化與摘要
# -----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CSIRSSI_DualHead_BN(num_classes=49, rssi_dim=4).to(device)

print("模型摘要:")
summary(model, input_data=(torch.randn(1, 1, 48).to(device), torch.randn(1, 4).to(device)))

模型摘要:


Layer (type:depth-idx)                   Output Shape              Param #
CSIRSSI_DualHead_BN                      [1, 49]                   --
├─Conv1d: 1-1                            [1, 64, 48]               256
├─BatchNorm1d: 1-2                       [1, 64, 48]               128
├─MaxPool1d: 1-3                         [1, 64, 24]               --
├─Conv1d: 1-4                            [1, 128, 24]              24,704
├─BatchNorm1d: 1-5                       [1, 128, 24]              256
├─MaxPool1d: 1-6                         [1, 128, 12]              --
├─Linear: 1-7                            [1, 128]                  196,736
├─Dropout: 1-8                           [1, 128]                  --
├─Linear: 1-9                            [1, 64]                   8,256
├─Dropout: 1-10                          [1, 64]                   --
├─Linear: 1-11                           [1, 128]                  640
├─BatchNorm1d: 1-12                      [1, 128]                  25

In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os



# 假設 COORDINATES 是一個 dict，將 class index 映射到 (x, y) 實座標
def compute_mean_distance_error(y_true, y_pred, coordinates):
    """
    y_true, y_pred: 一維的 NumPy 陣列，分別存放真實和預測的 label（整數）
    coordinates: dict, label -> (x, y)
    """
    errors = []

    for true_label, pred_label in zip(y_true, y_pred):
        # 取出對應的座標
        if true_label not in coordinates or pred_label not in coordinates:
            # 若某個 label 不在座標字典內，就跳過（或視需求處理）
            print(f"Label {true_label} or {pred_label} not in coordinates.")
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        # 計算歐氏距離
        error = np.linalg.norm(pred_coord - true_coord)
        errors.append(error)
    return np.mean(errors) , errors

COORDINATES = {
    # 下邊界 (1-10 和 40-31)
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
    # 左邊界 (1-11)
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
    # 上邊界 (11-21)
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),
    # 右邊界 (21-31)
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
    # 中間點 (41-49)
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

def labels_to_coords(label_tensor, coord_dict):
    coords = []
    for label in label_tensor:
        # 將 0-index 轉換成 1-index (例如 0 -> 1, 1 -> 2, ..., 48 -> 49)
        coords.append(coord_dict[label.item() + 1])
    return torch.tensor(coords, dtype=torch.float32, device=label_tensor.device)




criterion = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用裝置:", device)
alphas = np.arange(0.8, 1.1, 0.1)
num_runs = 10
epochs = 200
patience = 20

for alpha in alphas:
    test_accs = []
    test_mdes = []
    all_run_errors = []

    print(f"\n[Alpha = {alpha:.1f}] 開始 {num_runs} 次訓練")
    for run in range(1, num_runs + 1):
        print(f"Run {run}/{num_runs} - Alpha = {alpha:.1f}")
        model = CSIRSSI_DualHead_BN(num_classes=49, rssi_dim=4).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15)
        best_val_loss = float('inf')
        counter = 0

        for epoch in range(epochs):
            model.train()
            for csi_inputs, rssi_inputs, labels in train_loader:
                csi_inputs, rssi_inputs, labels = csi_inputs.to(device), rssi_inputs.to(device), labels.to(device)
                target_class = torch.argmax(labels, dim=1)
                optimizer.zero_grad()
                class_out, reg_out = model(csi_inputs, rssi_inputs)
                loss_cls = criterion(class_out, target_class)
                loss_reg = criterion_reg(reg_out, labels_to_coords(target_class, COORDINATES))
                loss = loss_cls + alpha * loss_reg
                loss.backward()
                optimizer.step()

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for csi_inputs, rssi_inputs, labels in val_loader:
                    csi_inputs, rssi_inputs, labels = csi_inputs.to(device), rssi_inputs.to(device), labels.to(device)
                    target_class = torch.argmax(labels, dim=1)
                    class_out, reg_out = model(csi_inputs, rssi_inputs)
                    loss_cls = criterion(class_out, target_class)
                    loss_reg = criterion_reg(reg_out, labels_to_coords(target_class, COORDINATES))
                    val_loss += (loss_cls + alpha * loss_reg).item() * csi_inputs.size(0)

            val_loss /= len(val_loader.dataset)
            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
                # 覆蓋存最佳模型
                torch.save(model.state_dict(), "repeat_2_4/rssicsi_reg_b_tmp.pth")
            else:
                counter += 1
                if counter >= patience:
                    break

        # === 測試階段：用最佳模型測試 ===
        model.load_state_dict(torch.load("repeat_2_4/rssicsi_reg_b_tmp.pth"))
        model.eval()
        all_true, all_pred = [], []
        with torch.no_grad():
            for csi_inputs, rssi_inputs, labels in test_loader:
                csi_inputs, rssi_inputs, labels = csi_inputs.to(device), rssi_inputs.to(device), labels.to(device)
                target_class = torch.argmax(labels, dim=1)
                class_out, _ = model(csi_inputs, rssi_inputs)
                pred = torch.argmax(class_out, dim=1)
                all_pred.extend(pred.cpu().numpy())
                all_true.extend(target_class.cpu().numpy())

        y_true = np.array(all_true) + 1
        y_pred = np.array(all_pred) + 1
        acc = 100 * np.mean(y_true == y_pred)
        mde, errors = compute_mean_distance_error(y_true, y_pred, COORDINATES)

        test_accs.append(acc)
        test_mdes.append(mde)
        all_run_errors.append(errors)
        print(f"✅ Run {run}: Acc = {acc:.2f}%, MDE = {mde:.4f}")

    # === 儲存 summary 與所有 error，檔名加 _b ===
    alpha_id = int(round(alpha * 10))
    folder_name = f"repeat_2_4/{alpha_id:02d}"
    os.makedirs(folder_name, exist_ok=True)

    # Summary per run
    df_runs = pd.DataFrame({
        'run': list(range(1, num_runs + 1)),
        'accuracy': test_accs,
        'mde': test_mdes
    })
    df_runs.to_csv(f"{folder_name}/csirssi_cls_reg_results{alpha_id:02d}_b.csv", index=False)

    # All errors (long format)
    error_records = []
    for run_idx, errors in enumerate(all_run_errors):
        for sample_idx, e in enumerate(errors):
            error_records.append({
                "run": run_idx + 1,
                "sample_idx": sample_idx + 1,
                "error": e
            })
    df_errors = pd.DataFrame(error_records)
    df_errors.to_csv(f"{folder_name}/csirssi_cls_reg_all_errors{alpha_id:02d}_b.csv", index=False)
    print(f"📁 Results & all errors saved to {folder_name}")

print("=== 所有 alpha 執行完畢 ===")


使用裝置: cuda

[Alpha = 0.8] 開始 10 次訓練
Run 1/10 - Alpha = 0.8
✅ Run 1: Acc = 98.08%, MDE = 0.0541
Run 2/10 - Alpha = 0.8
✅ Run 2: Acc = 97.91%, MDE = 0.0652
Run 3/10 - Alpha = 0.8
✅ Run 3: Acc = 98.40%, MDE = 0.0454
Run 4/10 - Alpha = 0.8
✅ Run 4: Acc = 97.58%, MDE = 0.0613
Run 5/10 - Alpha = 0.8
✅ Run 5: Acc = 97.87%, MDE = 0.0618
Run 6/10 - Alpha = 0.8
✅ Run 6: Acc = 97.58%, MDE = 0.0704
Run 7/10 - Alpha = 0.8
✅ Run 7: Acc = 97.58%, MDE = 0.0693
Run 8/10 - Alpha = 0.8
✅ Run 8: Acc = 97.99%, MDE = 0.0604
Run 9/10 - Alpha = 0.8
✅ Run 9: Acc = 97.95%, MDE = 0.0608
Run 10/10 - Alpha = 0.8
✅ Run 10: Acc = 98.20%, MDE = 0.0533
📁 Results & all errors saved to repeat_2_4/08

[Alpha = 0.9] 開始 10 次訓練
Run 1/10 - Alpha = 0.9
✅ Run 1: Acc = 97.83%, MDE = 0.0689
Run 2/10 - Alpha = 0.9
✅ Run 2: Acc = 97.46%, MDE = 0.0626
Run 3/10 - Alpha = 0.9
✅ Run 3: Acc = 97.95%, MDE = 0.0574
Run 4/10 - Alpha = 0.9
✅ Run 4: Acc = 98.12%, MDE = 0.0508
Run 5/10 - Alpha = 0.9
✅ Run 5: Acc = 97.54%, MDE = 0.0684
Run 6/

KeyboardInterrupt: 